In [1]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0

# Text Processing
import re
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, TFBertModel

# Machine Learning and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from albumentations import Compose, RandomCrop, HorizontalFlip, Normalize

# Miscellaneous
import os
from tqdm import tqdm
import glob
import random
from collections import defaultdict

from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


1.1. **Завантаження даних**

- Ознайомтесь із структурою датасету (зображення, текстові описи, цільова змінна).
- Визначте кількість пропущених даних, типи змінних (категоріальні, числові, текстові).

In [2]:
text_train_data = pd.read_csv('data/train.csv')
text_test_data = pd.read_csv('data/test.csv')

print(f"Number of missing TRAIN descriptions: {text_train_data['Description'].isnull().sum()}")
print(f"Number of missing TEST descriptions: {text_test_data['Description'].isnull().sum()}")

print(f"Number of unique TRAIN PetIDs: {text_train_data['PetID'].nunique()}")
print(f"Number of unique TEST PetIDs: {text_test_data['PetID'].nunique()}")

Number of missing TRAIN descriptions: 5
Number of missing TEST descriptions: 1
Number of unique TRAIN PetIDs: 6431
Number of unique TEST PetIDs: 1891


In [3]:
image_train_dir = 'data/images/images/train'
image_test_dir = 'data/images/images/test'

image_train_files = os.listdir(image_train_dir)
image_test_files = os.listdir(image_test_dir)

In [4]:
image_train_ids = [filename.split('-')[0] for filename in image_train_files]
image_test_ids = [filename.split('-')[0] for filename in image_test_files]

unique_train_image_pet_ids = set(image_train_ids)
print(f"Number of unique train PetIDs in images: {len(unique_train_image_pet_ids)}")

unique_test_image_pet_ids = set(image_test_ids)
print(f"Number of unique train PetIDs in images: {len(unique_test_image_pet_ids)}")

Number of unique train PetIDs in images: 6431
Number of unique train PetIDs in images: 1899


In [5]:
text_train_pet_ids = set(text_train_data['PetID'])
text_test_pet_ids = set(text_test_data['PetID'])

# Find PetIDs with images but no descriptions
pet_train_ids_with_images_only = unique_train_image_pet_ids - text_train_pet_ids
print(f"PetIDs TRAIN with images but no descriptions: {len(pet_train_ids_with_images_only)}")

# Find PetIDs with images but no descriptions
pet_test_ids_with_images_only = unique_test_image_pet_ids - text_test_pet_ids
print(f"PetIDs TEST with images but no descriptions: {len(pet_test_ids_with_images_only)}")

# Find PetIDs with descriptions but no images
pet_train_ids_with_descriptions_only = text_train_pet_ids - text_train_pet_ids
print(f"PetIDs TRAIN with descriptions but no images: {len(pet_train_ids_with_descriptions_only)}")

# Find PetIDs with descriptions but no images
pet_test_ids_with_descriptions_only = text_test_pet_ids - text_test_pet_ids
print(f"PetIDs TEST with descriptions but no images: {len(pet_test_ids_with_descriptions_only)}")

PetIDs TRAIN with images but no descriptions: 0
PetIDs TEST with images but no descriptions: 12
PetIDs TRAIN with descriptions but no images: 0
PetIDs TEST with descriptions but no images: 0


In [6]:
# Convert text PetIDs to sets
text_train_pet_ids = set(text_train_data['PetID'])
text_test_pet_ids = set(text_test_data['PetID'])

# Find PetIDs with images but no descriptions
train_image_ids_no_description = unique_train_image_pet_ids - text_train_pet_ids
test_image_ids_no_description = unique_test_image_pet_ids - text_test_pet_ids

# Output the results
print(f"Train images with no descriptions: {len(train_image_ids_no_description)}")
print(f"Test images with no descriptions: {len(test_image_ids_no_description)}")
print(f"IDs of Train images with no descriptions: {train_image_ids_no_description}")
print(f"IDs of Test images with no descriptions: {test_image_ids_no_description}")

Train images with no descriptions: 0
Test images with no descriptions: 12
IDs of Train images with no descriptions: set()
IDs of Test images with no descriptions: {'035992662', '2689341e7', '081301773', '515462e67', '02126e289', '2514503e7', '554965e66', '095314294', '670535e94', '867057e77', '7759517e2', '063521459'}


## Imputation

In [7]:
# Use the recommended approach to fill missing values
text_train_data['Description'] = text_train_data['Description'].fillna('No description provided')
text_test_data['Description'] = text_test_data['Description'].fillna('No description provided')


In [8]:
# List of missing PetIDs (already inspected as relevant)
missing_test_ids = list(test_image_ids_no_description)

# Create a DataFrame with placeholder descriptions
missing_test_data = pd.DataFrame({
    'PetID': missing_test_ids,
    'Description': ['No description provided'] * len(missing_test_ids)
})

# Append the new data to the test dataset
text_test_data = pd.concat([text_test_data, missing_test_data], ignore_index=True)

# Verify the updated dataset
print(f"Updated test data: {len(text_test_data)} rows")
print(f"Number of missing descriptions: {text_test_data['Description'].isnull().sum()}")


Updated test data: 1903 rows
Number of missing descriptions: 0


1.2. **Дослідження зображень**

- Перегляньте приклади фотографій тварин, їх розмір, роздільну здатність.
- Визначте, чи є декілька зображень для одного об’єкта (агрегація може знадобитися).

In [9]:
# List all image files
image_files = os.listdir(image_test_dir)

# Randomly select 5 images for inspection
sample_images = random.sample(image_files, 5)

# Display the images with their details
for image_file in sample_images:
    image_path = os.path.join(image_test_dir, image_file)
    with Image.open(image_path) as img:
        print(f"Image: {image_file}")
        print(f"Size: {img.size}, Format: {img.format}, Mode: {img.mode}")
        img.show()  # This opens the image in the default viewer

Image: 2d72ef0c4-2.jpg
Size: (360, 480), Format: JPEG, Mode: RGB
Image: 47ad7a82a-1.jpg
Size: (384, 480), Format: JPEG, Mode: RGB
Image: 7fe0a86f8-2.jpg
Size: (300, 400), Format: JPEG, Mode: RGB
Image: cc25fb529-1.jpg
Size: (266, 400), Format: JPEG, Mode: RGB
Image: 7d835cf7c-5.jpg
Size: (399, 263), Format: JPEG, Mode: RGB


In [10]:
# Initialize lists to store image dimensions
widths, heights = [], []

for image_file in image_files:
    image_path = os.path.join(image_test_dir, image_file)
    with Image.open(image_path) as img:
        widths.append(img.size[0])
        heights.append(img.size[1])

# Calculate statistics
print(f"Number of images: {len(image_files)}")
print(f"Average width: {sum(widths) / len(widths):.2f}")
print(f"Average height: {sum(heights) / len(heights):.2f}")
print(f"Minimum resolution: {min(widths)}x{min(heights)}")
print(f"Maximum resolution: {max(widths)}x{max(heights)}")

Number of images: 9448
Average width: 402.79
Average height: 390.01
Minimum resolution: 72x35
Maximum resolution: 1792x3184


In [11]:
# Create a dictionary to count images per PetID
image_count_per_pet = defaultdict(int)

for image_file in image_train_files:
    pet_id = image_file.split('-')[0]  # Extract the PetID
    image_count_per_pet[pet_id] += 1

# Count the number of PetIDs with multiple images
multiple_images = {pet_id: count for pet_id, count in image_count_per_pet.items() if count > 1}

print(f"Total PetIDs: {len(image_count_per_pet)}")
print(f"Number of PetIDs with multiple images: {len(multiple_images)}")

# Display some examples of multiple images per PetID
for pet_id, count in list(multiple_images.items())[:5]:
    print(f"PetID: {pet_id}, Number of images: {count}")


Total PetIDs: 6431
Number of PetIDs with multiple images: 5307
PetID: 2d725d001, Number of images: 3
PetID: a63364c39, Number of images: 6
PetID: ea055de86, Number of images: 26
PetID: 0db65104a, Number of images: 5
PetID: 76024f2ed, Number of images: 5


In [12]:
# Create a dictionary to count images per PetID
image_count_per_pet = defaultdict(int)

for image_file in image_test_files:
    pet_id = image_file.split('-')[0]  # Extract the PetID
    image_count_per_pet[pet_id] += 1

# Count the number of PetIDs with multiple images
multiple_images = {pet_id: count for pet_id, count in image_count_per_pet.items() if count > 1}

print(f"Total PetIDs: {len(image_count_per_pet)}")
print(f"Number of PetIDs with multiple images: {len(multiple_images)}")

# Display some examples of multiple images per PetID
for pet_id, count in list(multiple_images.items())[:5]:
    print(f"PetID: {pet_id}, Number of images: {count}")

Total PetIDs: 1899
Number of PetIDs with multiple images: 1687
PetID: bf9bd91e1, Number of images: 8
PetID: 16ffedcf8, Number of images: 7
PetID: 7a12a494f, Number of images: 9
PetID: d1870f34b, Number of images: 3
PetID: 856005eae, Number of images: 5


## Агрегація

In [13]:
class ImageDataset(Dataset):
    def __init__(self, pet_ids, image_dir, img_size=256, transform=None):
        self.pet_ids = pet_ids
        self.image_dir = image_dir
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        return len(self.pet_ids)

    def __getitem__(self, idx):
        pet_id = self.pet_ids[idx]
        file_path = f"{self.image_dir}/{pet_id}-1.jpg"

        if not os.path.exists(file_path):
            return pet_id, None  # Ensure exactly two elements are returned
        
        image = cv2.imread(file_path)
        if image is None:
            return pet_id, None  # Ensure exactly two elements are returned

        # Resize and pad the image
        image = resize_to_square(image, self.img_size)
        
        # Apply optional transforms
        if self.transform:
            image = self.transform(image)

        return pet_id, image  # Return a tuple of pet_id and image



In [14]:
def resize_to_square(im, img_size=256):
    old_size = im.shape[:2]
    ratio = float(img_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    color = [0, 0, 0]
    return cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

def extract_features(data_loader, model, output_file):
    features = {}  # Dictionary to store features

    for batch in tqdm(data_loader, desc="Processing batches"):
        valid_pet_ids, valid_images = [], []

        # Process each tuple in the batch
        for pet_id, image in batch:
            if image is not None:  # Filter out invalid images
                valid_pet_ids.append(pet_id)
                valid_images.append(image)

        if not valid_images:  # Skip if no valid images in the batch
            continue

        # Stack images into a batch and preprocess
        batch_images = np.array(valid_images)  # Convert to NumPy array
        batch_images = preprocess_input(batch_images)  # Preprocess for DenseNet

        # Extract embeddings
        batch_embeddings = model.predict(batch_images)

        # Save embeddings in the dictionary
        for pet_id, embedding in zip(valid_pet_ids, batch_embeddings):
            features[pet_id] = embedding

    # Convert features dictionary to DataFrame and save
    features_df = pd.DataFrame.from_dict(features, orient="index")
    features_df.to_csv(output_file, index_label="PetID")
    print("Extracting features has been finished")
    

# Custom collate function to return individual pairs
def collate_fn(batch):
    return [(item[0], item[1]) for item in batch]

In [15]:
# Train DataLoader
train_pet_ids = text_train_data['PetID'].unique()
train_dataset = ImageDataset(pet_ids=train_pet_ids, image_dir=image_train_dir)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)

# Test DataLoader
test_pet_ids = text_test_data['PetID'].unique()
test_dataset = ImageDataset(pet_ids=test_pet_ids, image_dir=image_test_dir)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)


In [16]:
# Create DenseNet model
inp = Input((256, 256, 3))
backbone = DenseNet121(input_tensor=inp, include_top=False)
x = GlobalAveragePooling2D()(backbone.output)
m = Model(inp, x)

In [17]:
# Extract and save train embeddings
extract_features(train_loader, m, "train_img_features.csv")

# Extract and save test embeddings
extract_features(test_loader, m, "test_img_features.csv")

Processing batches:   0%|          | 0/402 [00:00<?, ?it/s]2024-12-15 18:33:47.938156: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 1s/step


Processing batches:   0%|          | 1/402 [00:01<07:20,  1.10s/it]

1/1 [==============================] - 0s 354ms/step


Processing batches:   0%|          | 2/402 [00:01<04:32,  1.47it/s]

...

Processing batches:  98%|█████████▊| 117/119 [00:49<00:00,  2.36it/s]

1/1 [==============================] - 0s 386ms/step


Processing batches:  99%|█████████▉| 118/119 [00:50<00:00,  2.35it/s]

1/1 [==============================] - 0s 371ms/step


Processing batches: 100%|██████████| 119/119 [00:50<00:00,  2.35it/s]


In [21]:
img_train_features = pd.read_csv("train_img_features.csv")
img_test_features = pd.read_csv("test_img_features.csv")

In [23]:
# Merge with image features
merged_train_data = text_train_data.merge(img_train_features, on="PetID", how="left")
print(merged_train_data.head(1))

merged_test_data = text_test_data.merge(img_test_features, on="PetID", how="left")
print(merged_test_data.head(1))

       PetID                                        Description  \
0  d3b4f29f8  Mayleen and Flo are two lovely adorable sister...   

   AdoptionSpeed         0         1         2        3         4         5  \
0              2  0.000423  0.004769  0.003115  0.00386  0.083004  0.374298   

          6  ...      1014      1015      1016      1017      1018      1019  \
0  0.000869  ...  0.445891  0.227626  1.420346  0.654387  0.177218  1.166738   

       1020      1021      1022      1023  
0  0.971381  0.659021  0.245677  1.545983  

[1 rows x 1027 columns]
       PetID                                        Description         0  \
0  6697a7f62  This cute little puppy is looking for a loving...  0.000532   

          1         2         3         4         5         6         7  ...  \
0  0.004337  0.002847  0.002838  0.074986  0.262885  0.000559  0.001634  ...   

       1014      1015      1016      1017      1018      1019      1020  \
0  0.109452  0.107509  2.047832  0.653996